In [ ]:
import jsonlines

def get_span_noun_indices(cluster, all_words, constituents):
    noun_indices = []
    for span in cluster: 
        if [span[0], span[1], 'NP'] in constituents:
            noun_indices.append(span)
    if len(noun_indices) > 0:
        return noun_indices

def replace(all_words, span, mention):
    all_words[span[0]] = ['[['] + all_words_copy[span[0]:span[1]+1] + [']]'] + ['**'] + mention + ['**']
    for i in range(span[0]+1, span[1]+1):
        all_words[i] = ""
    return all_words

def is_containing_other_spans(span, all_spans):
    return any([s[0] >= span[0] and s[1] <= span[1] and s != span for s in all_spans])

def replace_corefs(all_words, clusters, constituents):
    all_spans = [span for cluster in clusters for span in cluster]
    for cluster in clusters:
        cluster.sort(key=lambda k: (k[0], k[1]))
        noun_indices = get_span_noun_indices(cluster, all_words, constituents)
        if noun_indices:  #if there is at least one noun phrase
            first_span = noun_indices[0]
        else: 
            first_span = cluster[0]
        mention_start, mention_end = first_span[0], first_span[1] + 1
        mention = all_words_copy[mention_start:mention_end]
        for span in cluster:
            if span != first_span and not is_containing_other_spans(span, all_spans):
                all_words = replace(all_words, span, mention)        
    return all_words

text = ""
def lst2str(word_list):
    global text
    special_signs = ["",",",".","!","?",":","'s"]
    for item in word_list:
        if type(item) == list:
            lst2str(item)
        else:
            if len(text) == 0 or item in special_signs:
                text += str(item)
            else:
                text += " " + str(item)
    return text

training_data = []
with jsonlines.open('train.english.jsonlines') as reader:
    for line in reader:
        text = ""
        all_words = []
        all_words_copy = []
        for sentence in line['sentences']:
            for word in sentence:
                all_words.append(word)
                all_words_copy.append(word)
        original_text = lst2str(all_words)
        text = ""
        new_text = lst2str(replace_corefs(all_words, line['clusters'], line['constituents']))
        training_data.append([original_text,new_text])

In [ ]:
import csv

header = ['source', 'target']
data = [[original_text,new_text] for [original_text,new_text] in training_data]
with open('dataset.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)